In [1]:
import os
import random
from glob import glob
import numpy as np

np.random.seed(0)
random.seed(0)

In [2]:
path_img = glob('/Users/tokkuman/git-tokkuman/seminiferous_tubule_stage_classification/datasets/images/*.tif')
with open('/Users/tokkuman/git-tokkuman/seminiferous_tubule_stage_classification/datasets/label_list.txt', 'r') as f:
    label_list = [line.rstrip() for line in f]

path_cls = {}
for i in label_list:
    path_cls[i] = []
for p in path_img:
    cls = os.path.basename(p)[os.path.basename(p).rfind('_')+1:os.path.basename(p).rfind('-')]
    if cls in label_list:
        path_cls[cls].append(p)

total = 0
for i in label_list:
    print(i, len(path_cls[i]))
    random.shuffle(path_cls[i])
    total += len(path_cls[i])
print('total: {}'.format(total))

1 101
2 104
3 101
4 102
5 104
6 105
7 102
8 101
9 103
10 103
11 101
12 102
total: 1229


In [3]:
len_test = 20
fold = 4
root = '/Users/tokkuman/git-tokkuman/seminiferous_tubule_stage_classification/datasets'
os.makedirs(os.path.join(root, 'split_list'), exist_ok=True)

with open(os.path.join(root, 'split_list', 'test.txt'), 'w') as f:
    for i in range(len_test):
        for c in label_list:
            f.write('{}\n'.format(os.path.basename(path_cls[c][0])))
            path_cls[c].pop(0)
       
total = 0
for i in label_list:
    total += len(path_cls[i])
print('total (train+validation): {}'.format(total))        

for fd in range(0, fold):
    dist = os.path.join(root, 'split_list', 'fold{}'.format(fd+1))
    os.makedirs(dist, exist_ok=True)
    with open(os.path.join(dist, 'train.txt'), 'w') as f:
        for c in label_list:
            #print(len(path_cls[c][:int(len(path_cls[c])/fold)*fd] + path_cls[c][int(len(path_cls[c])/fold)*(fd+1):]))
            for pa in path_cls[c][:int(len(path_cls[c])/fold)*fd] + path_cls[c][int(len(path_cls[c])/fold)*(fd+1):]:
                f.write('{}\n'.format(os.path.basename(pa)))

    with open(os.path.join(dist, 'validation.txt'), 'w') as f:
        for c in label_list:
            for pa in path_cls[c][int(len(path_cls[c])/fold)*fd:int(len(path_cls[c])/fold)*(fd+1)]:
                f.write('{}\n'.format(os.path.basename(pa)))

total (train+validation): 989


In [4]:
# trainとvalidationの被りなしを確認
for fd in range(0, fold):
    target = os.path.join(root, 'split_list', 'fold{}'.format(fd+1))
    with open(os.path.join(target, 'train.txt'), 'r') as f:
        filelist_train = [line.rstrip() for line in f]
    with open(os.path.join(target, 'validation.txt'), 'r') as f:
        filelist_validatioin = [line.rstrip() for line in f]
    print(len(np.unique(filelist_train + filelist_validatioin)))
    
filelist_validatioin_total = []
for fd in range(0, fold):
    target = os.path.join(root, 'split_list', 'fold{}'.format(fd+1))
    with open(os.path.join(target, 'validation.txt'), 'r') as f:
        filelist_validatioin = [line.rstrip() for line in f]
    filelist_validatioin_total += filelist_validatioin
print(len(np.unique(filelist_validatioin_total)))

989
989
989
989
972
